In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('pimadiabetes.csv')
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
X = df.iloc[:,:-1]
Y = df.iloc[:,-1:]


In [4]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

In [5]:
model = RandomForestClassifier()
model.fit(X_train,Y_train)

/tmp/ipykernel_7036/3555987548.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train,Y_train)


RandomForestClassifier()

In [6]:
Y_pred = model.predict(X_test)
accuracy_score(Y_test,Y_pred)

0.7207792207792207

In [7]:
from sklearn.model_selection import RandomizedSearchCV

In [8]:
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [9]:
param_dist = { 'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)],
              'criterion' : ['gini', 'entropy'],
              'max_depth' : [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)],
               'min_samples_split': [int(x) for x in np.linspace(start = 2, stop = 10, num = 2)],
              'max_features': ['auto','log2'], 
              'min_samples_leaf': [int(x) for x in np.linspace(start = 1, stop = 5, num = 5)],
              'random_state' : [int(x) for x in np.linspace(start = 21, stop = 45, num = 6)]              
                
}
param_dist

{'n_estimators': [100, 311, 522, 733, 944, 1155, 1366, 1577, 1788, 2000],
 'criterion': ['gini', 'entropy'],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
 'min_samples_split': [2, 10],
 'max_features': ['auto', 'log2'],
 'min_samples_leaf': [1, 2, 3, 4, 5],
 'random_state': [21, 25, 30, 35, 40, 45]}

In [10]:
classifierCV = RandomizedSearchCV(model,param_dist,cv=5)


In [11]:

classifierCV.fit(X_train,np.array(Y_train).ravel())

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100],
                                        'max_features': ['auto', 'log2'],
                                        'min_samples_leaf': [1, 2, 3, 4, 5],
                                        'min_samples_split': [2, 10],
                                        'n_estimators': [100, 311, 522, 733,
                                                         944, 1155, 1366, 1577,
                                                         1788, 2000],
                                        'random_state': [21, 25, 30, 35, 40,
                                                         45]})

In [12]:
bestEst = classifierCV.best_estimator_

In [13]:
classifierCV.best_params_

{'random_state': 21,
 'n_estimators': 1366,
 'min_samples_split': 2,
 'min_samples_leaf': 3,
 'max_features': 'auto',
 'max_depth': 20,
 'criterion': 'entropy'}

In [14]:
Y_pred1 = bestEst.predict(X_test)
accuracy_score(Y_test,Y_pred1)

0.7532467532467533

In [15]:
predictions = model.predict(X_test)
errors = abs(predictions - np.array(Y_test).ravel())
mape = 100 * np.mean(errors / len(np.array(Y_test).ravel()))
accuracy = 100 - mape
print('Model Performance')
print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
print('Accuracy = {:0.2f}%.'.format(accuracy))
# print(Y_test)

Model Performance
Average Error: 0.2792 degrees.
Accuracy = 99.82%.


In [18]:
predictions = bestEst.predict(X_test)
errors = abs(predictions - np.array(Y_test).ravel())
mape = 100 * np.mean(errors / len(np.array(Y_test).ravel()))
accuracy = 100 - mape
print('Model Performance')
print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
print('Accuracy = {:0.2f}%.'.format(accuracy))
# predictions

Model Performance
Average Error: 0.2468 degrees.
Accuracy = 99.84%.


In [25]:
# 
pred = bestEst.predict([[1,93,70,31,0,30.4,0.315,23]])
# {
#     'Pregnancies':1,
#     'Glucose':24,
#     'BloodPressure':65,
#     'SkinThickness':31,
#     'Insulin':0,
#     'BMI':24.8,
#     'DiabetesPedigreeFunction':40,
#     'Age':33
# }
pred

/home/leman/ML/lib/python3.8/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([0])